# `waterSpec` Demo

This notebook provides a detailed walkthrough of the `waterSpec` package using its primary high-level API, the `Analysis` class. We will see how to perform a full analysis in just a few lines of code, and then explore more advanced options.

## 1. Imports

In [ ]:
import os
import pprint

from IPython.display import Image

from waterSpec import Analysis

## 2. The Main Workflow

The `Analysis` class is the recommended entry point for most users. You initialize it with your data file and column names, and it handles all the preprocessing. Then, you call `run_full_analysis()` to perform the complete workflow and generate outputs.

In [ ]:
# Define paths for our data and where to save outputs
file_path = 'sample_data.csv'
output_dir = 'demo_output'

# 1. Initialize the Analysis object.
# Using verbose=True provides detailed step-by-step logging.
analyzer = Analysis(
    file_path=file_path,
    time_col='timestamp',
    data_col='concentration',
    param_name='Sample Concentration',
    verbose=True
)

# 2. Run the full analysis workflow.
# This performs model fitting (auto-selecting standard vs. segmented),
# detects significant peaks, and saves all outputs.
results = analyzer.run_full_analysis(output_dir=output_dir, seed=42)

print(f"\nAnalysis complete. Outputs saved to '{output_dir}'.")

## 3. Inspecting the Results

The `run_full_analysis` method returns a dictionary containing all results, including the fitted parameters, confidence intervals, and the automated interpretation.

In [ ]:
# The returned dictionary contains all results.
print("--- Full Analysis Results ---")
pprint.pprint(results)

# You can directly access the generated summary text.
print("\n--- Summary Text ---")
print(results['summary_text'])

### Checking for Uncertainty

A key feature of `waterSpec` is its transparent reporting of uncertainty. You can programmatically check for any issues flagged during the analysis.

In [ ]:
# The `uncertainty_warnings` key contains a list of any warnings generated.
if results['uncertainty_warnings']:
    print("\n--- Uncertainty Warnings Detected ---")
    for warning in results['uncertainty_warnings']:
        print(f"- {warning}")
else:
    print("\n--- No Uncertainty Warnings Detected ---")

## 4. Visualizing the Output

The analysis automatically generates and saves a plot of the spectrum and the fit. We can display it directly in the notebook.

In [ ]:
# Construct the path to the output plot
sanitized_name = analyzer._sanitize_filename(analyzer.param_name)
plot_path = os.path.join(output_dir, f"{sanitized_name}_spectrum_plot.png")

# Display the image
if os.path.exists(plot_path):
    display(Image(filename=plot_path))
else:
    print(f"Plot not found at {plot_path}")